Desarrollo de Aplicaciones en PySpark
===

* *30 min* | Última modificación: Junio 22, 2019

En este tutorial se describe como implementar y ejecutar una aplicación usando PySpark. Al finalizar este tutorial, el lector estará en capacidad de:

* Describir el proceso general de desarrollo de una aplicación.

* Gestionar los archivos de entrada y salida de la aplicación.

* Ejecutar la aplicación en Spark.

## Descripción de la aplicación

La aplicación desarrollada será el conteo de frecuencia de palabras, desarrollado en el tutorial 'WordCount en Spark'. 

## Preparación de los datos

En este proceso, se que los datos originales se encuentran en una carpeta de la máquina local del usuario. Para este ejemplo, se crea el directorio `wordcount`en la carpeta actual y se crean tres archivos dentro de él.

In [1]:
#
# Creación de la carpeta wordcount en la máquina local.
#
!mkdir -p wordcount/

A continuación se crean los tres archivos de prueba.

In [2]:
%%writefile wordcount/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing wordcount/text0.txt


In [3]:
%%writefile wordcount/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing wordcount/text1.txt


In [4]:
%%writefile wordcount/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing wordcount/text2.txt


## Copia de los datos de entrada al sistema HDFS

En esta aplicación se supone que los datos siempre estarán en la carpeta `wordcount` del directorio actual de trabajo de la máquina local. El primer paso consisten en mover los archivos de la máquina local al sistema HDFS. Por ahora, este paso se hará manualmente. Se define que la aplicación usará siempre la carpeta `/tmp/wordcount/` del sistema HDFS.

In [5]:
#
# Se crea la carpeta /tmp/wc en el sistema HDFS.
#
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -mkdir /tmp/wordcount/input

In [6]:
#
# Copia los archvios del directorio local wordcount/
# al directorio /tmp/wordcount/input en el hdfs
#
!hdfs dfs -copyFromLocal wordcount/* /tmp/wordcount/input/

In [7]:
#
# Verifica que los archivos esten copiados
# en el hdfs
#
!hdfs dfs -ls /tmp/wordcount/input

Found 3 items
-rw-r--r--   1 root supergroup       1093 2019-11-15 00:38 /tmp/wordcount/input/text0.txt
-rw-r--r--   1 root supergroup        352 2019-11-15 00:38 /tmp/wordcount/input/text1.txt
-rw-r--r--   1 root supergroup        440 2019-11-15 00:38 /tmp/wordcount/input/text2.txt


## Implementación del programa en PySpark

El archivo `wordcount.py` contiene la implementación de la aplicación. El código es el siguiente:

In [8]:
%%writefile wordcount.py

import findspark
from pyspark import SparkConf, SparkContext
from operator import add

APP_NAME = "wordcount-app"

findspark.init()
conf = SparkConf().setAppName(APP_NAME) 
sc = SparkContext(conf=conf)

# Lee los archivos de la carpeta de entrada
text = sc.textFile("/tmp/wordcount/input/*.txt")

# Este es el algoritmo para el conteo de frecuencia
words = text.flatMap(lambda x: x.split())
wc = words.map(lambda x: (x,1))
counts = wc.reduceByKey(add)

# Escribe los resultados en la carpeta de salida.
counts.saveAsTextFile("/tmp/wordcount/output")

Writing wordcount.py


## Ejecución de la aplicación

In [9]:
#
# La aplicación es ejecutada usando spark-submit,
# el cual ejecuta el programa wordcount.py en Spark
#
!spark-submit wordcount.py

19/11/15 00:38:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
19/11/15 00:38:35 INFO spark.SparkContext: Running Spark version 2.4.4
19/11/15 00:38:35 INFO spark.SparkContext: Submitted application: wordcount-app
19/11/15 00:38:35 INFO spark.SecurityManager: Changing view acls to: root
19/11/15 00:38:35 INFO spark.SecurityManager: Changing modify acls to: root
19/11/15 00:38:35 INFO spark.SecurityManager: Changing view acls groups to: 
19/11/15 00:38:35 INFO spark.SecurityManager: Changing modify acls groups to: 
19/11/15 00:38:35 INFO spark.SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(root); groups with view permissions: Set(); users  with modify permissions: Set(root); groups with modify permissions: Set()
19/11/15 00:38:36 INFO util.Utils: Successfully started service 'sparkDriver' on port 36081.
19/11/15 00:38:36 INFO spark.SparkEnv

## Archivos de resultados

La carpeta `/tmp/wordcount/output` contiene los resultados de la ejecución del programa.

In [10]:
# Archivos con los resultados. Note que se 
# generan varios archivos de resultados.
!hdfs dfs -ls /tmp/wordcount/output/

Found 5 items
-rw-r--r--   1 root supergroup          0 2019-11-15 00:38 /tmp/wordcount/output/_SUCCESS
-rw-r--r--   1 root supergroup        778 2019-11-15 00:38 /tmp/wordcount/output/part-00000
-rw-r--r--   1 root supergroup        562 2019-11-15 00:38 /tmp/wordcount/output/part-00001
-rw-r--r--   1 root supergroup        510 2019-11-15 00:38 /tmp/wordcount/output/part-00002
-rw-r--r--   1 root supergroup        594 2019-11-15 00:38 /tmp/wordcount/output/part-00003


El archivo `output/_SUCCESS` es un archivo vacio que indica que el programa fue ejecutado correctamente.

In [11]:
!hdfs dfs -cat /tmp/wordcount/output/part-00000 | head

('interpretation,', 1)
('of', 8)
('in', 5)
('data.', 1)
('Especially', 1)
('analytics', 8)
('simultaneous', 1)
('operations', 1)
('research', 2)
('quantify', 1)


## Archivo de comandos del sistema operativo

Finalmente, se crea un script que copie los archivos y ejecute la aplicación.

In [12]:
%%writefile wordcountapp
#! /bin/bash

# borra la carpeta input si existe
!hdfs dfs -rm -r -f /tmp/wordcount/input

# crea la carpeta
!hdfs dfs -mkdir /tmp/wordcount/input

# copia los archivos de entrada de la 
# maquina local al sistema hdfs
!hdfs dfs -copyFromLocal wordcount/* /tmp/wordcount/input/

# ejecuta la aplicación de spark
spark-submit wordcount.py

Writing wordcountapp


Esta aplicación sería ejecutada en Terminal con el siguiente comando:

    $ bash wordcountapp

---

**Limpieza de las carpetas de trabajo**

In [13]:
!rm wordcountapp
!rm -rf wordcount
!hdfs dfs -rm -r -f /tmp/wordcount/

Deleted /tmp/wordcount
